In [47]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, regularizers

import matplotlib.pyplot as plt

import numpy as np
import scipy.sparse as sp
from sklearn.preprocessing import LabelBinarizer



# Settings / Hyperparams
dataset = "facebook.npz"
epochs = 100

# Read the dataset
fb_data = np.load(dataset)

## Extract data
# Number of Nodes
nodes = len(fb_data['target'])

# Feature list
X = fb_data['features']
# Normalise
X /= X.sum(1).reshape(-1, 1)

# Edge list
edges = fb_data['edges']
# Convert edge list to sparse edge coordinate matrix
A = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                    shape=(nodes, nodes), dtype=np.float32)

# One-hot encode labels
lb = LabelBinarizer()
labels = lb.fit_transform(fb_data['target'])

In [48]:
# Preprocess adjacency matrix by adding identity and dot with degree matrix
AStar = A + sp.eye(A.shape[0])
D = sp.diags(np.power(np.array(AStar.sum(1)), -0.5).flatten())
AStar = D.dot(AStar).dot(D).tocsr()

graph = [X, AStar]

In [50]:
y_train = np.zeros(labels.shape, dtype=np.int32)
y_val = np.zeros(labels.shape, dtype=np.int32)
y_test = np.zeros(labels.shape, dtype=np.int32)

train_range = range(int(nodes/5))
val_range = range(int(nodes/5), int(2*nodes/5))
test_range = range(int(2*nodes/5), nodes)

y_train[train_range] = labels[train_range]
y_val[train_range] = labels[train_range]
y_test[train_range] = labels[train_range]

training_mask = np.zeros(labels.shape[0])
training_mask[train_range] = 1
training_mask = np.array(training_mask, dtype=np.bool)